In [0]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jan  7 14:48:11 2020

@author: basit
"""
#%%
from __future__ import print_function, division
import os
import numpy as np
import pandas as pd
from PIL import Image  #image.op
import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
#import torchvision
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
# Ignore warnings
import torch.nn as nn
import torch.optim as optim
#from torch.autograd import Variable
#import torch.nn.functional as F
#import torch.nn as nn
#import torch.optim as optim
import pickle



In [9]:

wordC = pd.read_hdf("/content/drive/My Drive/Colab Notebooks/eee443_project_dataset_train.h5", 'word_code')
with open('/content/drive/My Drive/Colab Notebooks/invalidAllUrls', 'rb') as f:
    invUrls = pickle.load(f)
with open('/content/drive/My Drive/Colab Notebooks/cleanTrainID', 'rb') as f:
    trainID = pickle.load(f)
with open('/content/drive/My Drive/Colab Notebooks/cleanTrainCap', 'rb') as f:
    trainCap = pickle.load(f)

w = []
for col in wordC.columns:
    w.append( col)

w = np.asarray(w)[np.newaxis]
wCodes = wordC.to_numpy().astype(int)
wordC = wordC.to_dict('split')
wordDict = dict(zip(wordC['data'][0], wordC['columns']))

/usr/local/lib/python3.6/dist-packages/tables/path.py:118: NaturalNameWarning: object name is a Python keyword: 'from'; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  % (name, _warnInfo), NaturalNameWarning)
/usr/local/lib/python3.6/dist-packages/tables/path.py:118: NaturalNameWarning: object name is a Python keyword: 'and'; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  % (name, _warnInfo), NaturalNameWarning)
/usr/local/lib/python3.6/dist-packages/tables/path.py:118: NaturalNameWarning: object name is a Python keyword: 'is'; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  % (name, _warnInfo), NaturalNameWarning)
/usr/local/lib/python3.6/dist-packages/tables/path.py:118: NaturalNameWarning: object name is a Python keyword: 'in'; you will not be able to use natural naming to access this object; 

In [0]:

    

##explaination of the photo
#photo2Explain = 20
#for n in trainCap[photo2Explain]:
#     if n == 0:
#         break
#     else:
#         print(w[np.where(wCodes == n)])
##show the explained photo
#imgDir = 'F:/neural/images/img' + str(trainID[photo2Explain] - 1) + '.jpg'
#pix = mpimg.imread(imgDir)
#plt.imshow(pix)

#%%
root = '/content/drive/My Drive/Colab Notebooks/'
totalList = sorted(os.listdir(root+'images/'), key=len)
lens = [int(len(totalList)*0.7), int(len(totalList)*0.2), int(len(totalList)*0.1)+1]
train, test, val = torch.utils.data.dataset.random_split(totalList, lens)
pp = ""


dataset_tr   = ImageDataset(root_dir = root, data = train, transform = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.Resize((224,224),interpolation=Image.NEAREST), transforms.ToTensor()]))
dataset_test = ImageDataset(root_dir = root, data = test, transform = transforms.Compose([transforms.Resize((224,224),interpolation=Image.NEAREST), transforms.ToTensor()]))
dataset_val  = ImageDataset(root_dir = root, data = val, transform = transforms.Compose([transforms.Resize((224,224),interpolation=Image.NEAREST), transforms.ToTensor()]))

dataloader_train = DataLoader(dataset_tr, batch_size=16, shuffle=True)
dataloader_test  = DataLoader(dataset_test, batch_size=1, shuffle=False)
dataloader_val   = DataLoader(dataset_val, batch_size=16, shuffle=False)

In [0]:
#%%
class ImageDataset(Dataset):
    """MIT image dataset."""
    
    def __init__(self, root_dir, data, transform = None):
        self.root_dir  = root
        self.dataPtr   = data
        self.transform = transform
        with open(self.root_dir + 'cleanTrainCap', 'rb') as f:
            self.trainCap = pickle.load(f)  
        with open('cleanTrainID', 'rb') as f:
            self.trainID = pickle.load(f) - 1

    def __len__(self):
        return len(self.dataPtr)

    def __getitem__(self, idx):
        img_name = self.root_dir + 'images/' + self.dataPtr[idx]
        image    = Image.open(img_name)
#        image.show()
        if self.transform:
            image = self.transform(image)
        if image.size()[0] == 1:
            image = image.repeat(3,1,1)
#            plt.figure()
#            plt.imshow(  image.permute(1, 2, 0)  )
        elif  image.size()[0] == 4:
            image = image[0].repeat(3,1,1)
            
        imgNum = int(self.dataPtr[idx][3:-4])
        captions = self.trainCap[self.trainID == imgNum]
        rand    = np.random.randint(captions.shape[0])
        caption = captions[rand]
        
        sample = {'image': image, 'caption': torch.from_numpy(np.array(caption))}
        return sample